# Homework 4: Language Modelling in Hangman

Student Name: NA CHANG

Student ID: 858604

Python version used: 2.7

## General info

<b>Due date</b>: 11pm, Wednesday May 2nd

<b>Submission method</b>: see LMS

<b>Submission materials</b>: completed copy of this iPython notebook

<b>Late submissions</b>: -20% per day

<b>Marks</b>: 5% of mark for class

<b>Overview</b>: In this homework, you'll be creating an 'artificial intelligence' player for the classic Hangman word guessing game. You will need to implement several different automatic strategies based on character level language models, ranging from unigram approaches to higher over n-gram models. Your objective is to create an automatic player which makes the fewest mistakes.

<b>Materials</b>: See the main class LMS page for information on the basic setup required for this class, including an iPython notebook viewer and the python packages NLTK, Numpy, Scipy, Matplotlib, Scikit-Learn, and Gensim. In particular, if you are not using a lab computer which already has it installed, we recommend installing all the data for NLTK, since you will need various parts of it to complete this assignment. You can also use any Python built-in packages, but do not use any other 3rd party packages; if your iPython notebook doesn't run on the marker's machine, you will lose marks.  

<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). You should leave the output from running your code in the iPython notebook you submit, to assist with marking. The amount each section is worth is given in parenthesis after the instructions. You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it.

<b>Extra credit</b>: Each homework has a task which is optional with respect to getting full marks on the assignment, but that can be used to offset any points lost on this or any other homework assignment (but not the final project or the exam). We recommend you skip over this step on your first pass, and come back if you have time: the amount of effort required to receive full marks (1 point) on an extra credit question will be substantially more than earning the same amount of credit on other parts of the homework.

<b>Updates</b>: Any major changes to the assignment will be announced via LMS. Minor changes and clarifications will be announced in the forum on LMS, we recommend you check the forum regularly.

<b>Academic Misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place.


## The Hangman Game

The <a href="https://en.wikipedia.org/wiki/Hangman_(game)">Hangman game</a> is a simple game whereby one person thinks of a word, which they keep secret from their opponent, who tries to guess the word one character at a time. The game ends when the opponent makes more than a fixed number of incorrect guesses, or they figure out the secret word before then (in which case they *win*). 

Here's a simple version of the game, and a method allowing interactive play. 

In [1]:
# allowing better python 2 & python 3 compatibility 
from __future__ import print_function 

def hangman(secret_word, guesser, max_mistakes=8, verbose=True, **guesser_args):
    """
        secret_word: a string of lower-case alphabetic characters, i.e., the answer to the game
        guesser: a function which guesses the next character at each stage in the game
            The function takes a:
                mask: what is known of the word, as a string with _ denoting an unknown character
                guessed: the set of characters which already been guessed in the game
                guesser_args: additional (optional) keyword arguments, i.e., name=value
        max_mistakes: limit on length of game, in terms of allowed mistakes
        verbose: be chatty vs silent
        guesser_args: keyword arguments to pass directly to the guesser function
    """
    secret_word = secret_word.lower()
    mask = ['_'] * len(secret_word)
    guessed = set()
    if verbose:
        print("Starting hangman game. Target is", ' '.join(mask), 'length', len(secret_word))
    
    mistakes = 0
    while mistakes < max_mistakes:
        if verbose:
            print("You have", (max_mistakes-mistakes), "attempts remaining.")
        guess = guesser(mask, guessed, **guesser_args)

        if verbose:
            print('Guess is', guess)
        if guess in guessed:
            if verbose:
                print('Already guessed this before.')
            mistakes += 1
        else:
            guessed.add(guess)
            if guess in secret_word:
                for i, c in enumerate(secret_word):
                    if c == guess:
                        mask[i] = c
                if verbose:
                    print('Good guess:', ' '.join(mask))
            else:
                if verbose:
                    print('Sorry, try again.')
                mistakes += 1
                
        if '_' not in mask:
            if verbose:
                print('Congratulations, you won.')
            return mistakes
        
    if verbose:
        print('Out of guesses. The word was', secret_word)    
    return mistakes

def human(mask, guessed, **kwargs):
    """
    simple function for manual play
    """
    print('Enter your guess:')
    try:
        return raw_input().lower().strip() # python 3
    except NameError:
        return input().lower().strip() # python 2

You can play the game interactively using the following command:

In [ ]:
hangman('whatever', human, 8, True)

<b>Instructions</b>: We will be using the words occurring in the *Brown* corpus for *training* an artificial intelligence guessing algorithm, and for *evaluating* the quality of the method. Note that we are intentionally making the hangman game hard, as the AI will need to cope with test words that it has not seen before, hence it will need to learn generalisable patterns of characters to make reasonable predictions.

Your first task is to compute the unique word types occurring in the *Brown* corpus, using `nltk.corpus.Brown`, selecting only words that are entirely comprised of alphabetic characters, and lowercasing the words. Finally, randomly shuffle (`numpy.random.shuffle`) this collection of word types, and split them into disjoint training and testing sets. The test set should contain 1000 word types, and the rest should be in the training set. Your code should print the sizes of the training and test sets.

Feel free to test your own Hangman performance using `hangman(numpy.random.choice(test_set), human, 8, True)`. It is surprisingly difficult (and addictive)!

(0.5 mark)

In [2]:
import nltk
import numpy

nltk.download('brown')
from nltk.corpus import brown

words = set(brown.words())

word_type = []
for wr in words:
    if wr.isalpha():
        word_type.append(wr.lower())

numpy.random.shuffle(word_type)
test_set = word_type[:1000]
training_set = word_type[1000:]

print ("Length of test set: ", len(test_set)) #1000
print ("Length of training set: ", len(training_set)) #45185

[nltk_data] Downloading package brown to /Users/Rena/nltk_data...
[nltk_data]   Package brown is already up-to-date!
Length of test set:  1000
Length of training set:  45185


<b>Instructions</b>: To set a baseline, your first *AI* attempt will be a trivial random method. For this you should implement a guessing method, similar to the `human` method above, i.e., using the same input arguments and returning a character. Your method should randomly choose a character from the range `'a'...'z'` after excluding the characters that have already been guessed in the current game (all subsequent AI approaches should also exclude previous guesses). You might want to use `numpy.random.choice` for this purpose.

To measure the performance of this (and later) techiques, implement a method that measures the average number of mistakes made by this technique over all the words in the `test_set`. You will want to turn off the printouts for this, using the `verbose=False` option, and increase the cap on the game length to `max_mistakes=26`. Print the average number of mistakes for the random AI, which will become a baseline for the following steps.

(1 mark)

In [3]:
def evaluate(agent, **guesser_args):
    mis_count = 0
    for word in test_set:
        mistake = hangman(word, agent, 26, verbose = False, **guesser_args)
        mis_count += mistake
    return mis_count



def random_agent(mask, guessed, **kwargs):
    char_pool = map(chr, range(ord('a'),ord('z')+1,1))
    map(char_pool.remove, guessed)
    response = numpy.random.choice(char_pool)
    return response

print ("AVG mistake-random agent: ", float(evaluate(random_agent))/len(test_set))

AVG mistake-random agent:  16.714


**Instructions:** As your first real AI, you should train a *unigram* model over the training set.  This requires you to find the frequencies of characters over all training words. Using this model, you should write a guess function that returns the character with the highest probability, after aggregating (summing) the probability of each blank character in the secret word. Print the average number of mistakes the unigram method makes over the test set. Remember to exclude already guessed characters, and use the same evaluation method as above, so the results are comparable. (Hint: it should be much lower than for random).

(1 mark)

In [4]:
def char_count(corpus):
    char_pool = map(chr, range(ord('a'),ord('z')+1,1))
    char_freq = {}
    for char in char_pool:
        char_freq[char] = 0

    for word in corpus:
        for char in word:
            char_freq[char] += 1
    return char_freq

def unigram_agent(mask, guessed, char_freq):
    freq = dict(char_freq)
    map(freq.pop, guessed)
    char_sort = sorted(freq.items(), key = lambda x: x[1], reverse = True)
    (c, f) = char_sort[0]
    return c

uni_freq = char_count(training_set)

print ("AVG mistake-unigram agent: ", float(evaluate(unigram_agent, char_freq=uni_freq))/len(test_set))

AVG mistake-unigram agent:  10.368


**Instructions:** The length of the secret word is an important clue that we might exploit. Different length words tend to have different distributions over characters, e.g., short words are less likely to have suffixes or prefixes. Your job now is to incorporate this idea by conditioning the unigram model on the length of the secret word, i.e., having *different* unigram models for each length of word. You will need to be a little careful at test time, to be robust to the (unlikely) situation that you encounter a word length that you didn't see in training. Create another AI guessing function using this new model, and print its test performance.   

(0.5 marks)

In [5]:
def len_char_count(corpus = training_set):
    len_char_freq = {}

    char_pool = map(chr, range(ord('a'),ord('z')+1,1))
    default_char_freq = {}
    for char in char_pool:
        default_char_freq[char] = 0

    counter = 0
    for word in corpus:
        exist_dic = len_char_freq.setdefault(len(word), dict(default_char_freq))

        new_dic = char_count(word)
        for key in exist_dic.keys():
            exist_dic[key] += new_dic[key]
    return len_char_freq 

def len_unigram_agent(mask, guessed, char_freq):

    keys = char_freq.keys()
    word_length = len(mask)
    if word_length in keys:
        len_char_freq = dict(char_freq[word_length])
    else:
        len_char_freq = dict(char_freq[keys[-1]])
    return unigram_agent(mask, guessed, len_char_freq)

len_unigram_freque = len_char_count()


print ("AVG mistake-length unigram agent: ", float(evaluate(len_unigram_agent, char_freq=len_unigram_freque))/len(test_set))

AVG mistake-length unigram agent:  10.28


**Instructions:** Now for the main challenge, using a *ngram* language model over characters. The order of characters is obviously important, yet this wasn't incorporated in any of the above models. Knowing that the word has the sequence `n _ s s` is a pretty strong clue that the missing character might be `e`. Similarly the distribution over characters that start or end a word are highly biased (e.g., toward common prefixes and suffixes, like *un-*, *-ed* and *-ly*).

Your job is to develop a *ngram* language model over characters, train this over the training words (being careful to handle the start of each word properly, e.g., by padding with sentinel symbols.) You should use linear interpolation to smooth between the higher order and lower order models, and you will have to decide how to weight each component. 

Your guessing AI algorithm should apply your language model to each blank position in the secret word by using as much of the left context as is known. E.g., in `_ e c _ e _ _` we know the full left context for the first blank (context=start of word), we have a context of two characters for the second blank (context=ec), one character for the second last blank (context=e), and no known context for the last one. If we were using a *n=3* order model, we would be able to apply it to the first and second blanks, but would only be able to use the bigram or unigram distributions for the subsequent blanks. As with the unigram model, you should sum over the probability distributions for each blank to find the expected count for each character type, then select the  character with the highest expected count.

Implement the ngram method for *n=3,4* and *5* and evaluate each of these three models over the test set. Do you see any improvement over the unigram methods above?

(2 marks)

In [7]:
def word_padding(corpus, padding):
    prefix = ''
    suffix = ''
    for i in range(padding-1):
        num = i+1
        suffix += str(num)
        prefix = str(num)+prefix

    padded_corpus = []
    for word in corpus:
        w = prefix + word + suffix
        padded_corpus.append(w)

    return padded_corpus


def n_gram_count(corpus, n):
    n_gram_freq = {}
    for word in corpus:
        if len(word) < n:
            pass
        else:
            while len(word) >= n:
                key = word[:n]
                n_gram_freq[key] = n_gram_freq.setdefault(key, 0) + 1
                word = word[1:]
    return n_gram_freq

def n_gram_prob(lower_gram_freq, higher_grame_freq, corpus_word_no):
    n_gram_prob = {}
    for key, val in higher_grame_freq.iteritems():
        
        if key[:-1] in lower_gram_freq:
            n_gram_prob[key] = float(val) / lower_gram_freq[key[:-1]]
        else:
            
            n_gram_prob[key] = float(val) / corpus_word_no
    return n_gram_prob

unigram_prob = {}
unigram_count = len(''.join(training_set))
for key, val in uni_freq.iteritems():
    unigram_prob[key] = float(val)/unigram_count


bigram_freq = n_gram_count(word_padding(training_set, 2),2)
trigram_freq = n_gram_count(word_padding(training_set, 3),3)
fourgram_freq = n_gram_count(word_padding(training_set, 4),4)
fivegram_freq = n_gram_count(word_padding(training_set, 5),5)



word_count = len(training_set)
bigram_prob = n_gram_prob(uni_freq, bigram_freq, word_count)
trigram_prob = n_gram_prob(bigram_freq, trigram_freq, word_count)
fourgram_prob = n_gram_prob(trigram_freq, fourgram_freq, word_count)
fivegram_prob = n_gram_prob(fourgram_freq, fivegram_freq, word_count)


ngram_proba = [unigram_prob, bigram_prob, trigram_prob, fourgram_prob, fivegram_prob]

def n_gram_agent(mask, guessed, weight, ngram_prob):
    n = len(weight)
    ngram_prob = list(ngram_prob)
    mask = word_padding([''.join(mask)], n)[0]
    index = 0

    probability = {}
    char_pool = map(chr, range(ord('a'),ord('z')+1,1))

    while index < len(mask):
        if mask[index] == '_':
            context = mask[:index]
            for char in char_pool:
                if char not in guessed:
                    prob = 0
                    gram = char
                    for i in range(n):
                        
                        if len(context) < i:
                            break
                        try:
                            prob += ngram_prob[i][gram] * weight[i]
                        except:
                            pass
                        gram = context[-(i+1):]+char
                    probability[char] = prob + probability.setdefault(char, 0)
        index += 1

    char_sort = sorted(probability.items(), key = lambda x: x[1], reverse = True)
    (c, f) = char_sort[0]
    
    return c



bigram_weight = [0.3,0.7]
trigram_weight = [0.2, 0.3, 0.5]
fourgram_weight = [0.05, 0.15, 0.2, 0.6]
fivegram_weight = [0.05, 0.05, 0.1, 0.2, 0.6]


print ("AVG mistake trigram agent: ", float(evaluate(n_gram_agent, weight = trigram_weight, ngram_prob = ngram_proba))/len(test_set))

print ("AVG mistake fourgram agent: ", float(evaluate(n_gram_agent, weight = fourgram_weight, ngram_prob = ngram_proba))/len(test_set))

print ("AVG mistake fivegram agent: ", float(evaluate(n_gram_agent, weight = fivegram_weight, ngram_prob = ngram_proba))/len(test_set))

AVG mistake trigram agent:  8.4
AVG mistake fourgram agent:  7.901
AVG mistake fivegram agent:  7.684


## Bonus: Improving the AI

**Instructions:** To get the extra credit, you should try to develop a more effective AI for hangman. Feel free to engage your creativity here! Possibilities include better conditioning on the length of the word and the parts that are known, fancier smoothing methods, using backwards ngram models, or a fancier inference algorithm like forward-back using a HMM. Ensure you report the test performance of your method.

You will be marked based on the ambition of your approach and on its accuracy. If you have tried some truly spectacular method but it didn't really work, then please include your implementation and an explanation, which will still attract marks for ambition.

(1 bonus mark) 